In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Input the datasets and see the shape etc**

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print(train.shape)
print(test.shape)

In [ ]:
train.head(5)

In [ ]:
test.head(5)

Sales Price is the column that we need to predict

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn
print(train.iloc[:,-1].describe())
seaborn.distplot(train['SalePrice'])
train.iloc[:,-1].isnull().sum()


In [ ]:
#qqplot
from scipy import stats
import pylab
stats.probplot(train.iloc[:,-1],dist="norm",plot=pylab)
pylab.show

the above seaborn.distpot i found online :). Anyways, it can be seen that the saleprice is slightly right scewed but that was expected. most of the people take lower priced accomodations and there is a limit at price 0 too. They number of extremely high prices is quite low. They probably are the houses that rich people bought and arent exactly outliers. And there are no missing values atleast not in this column that needs to be predicted.

In [ ]:
train.describe(include='all')

Finding and imputing all the columns with missing values

In [ ]:
#Finding all columns with null values
train.columns[train.isnull().any()].tolist()

In [ ]:
#Function for Finding and replacing mmissing values with median
def num_miss(df_in, col_name):
    m = df_in[col_name].describe()['50%']
    df_in.loc[(df_in[col_name].isnull()),col_name] = m
    return df_in


In [ ]:
num_miss(train, 'LotFrontage')
num_miss(train, 'MasVnrArea')
num_miss(train, 'GarageYrBlt')
train.columns[train.isnull().any()].tolist()

In [ ]:
#Function for Finding and replacing mmissing values with most appearing value
def str_miss(df_in, col_name):
    m = df_in[col_name].describe()['top']
    df_in.loc[(df_in[col_name].isnull()),col_name] = m
    return df_in


In [ ]:
str_miss(train,'Alley')
str_miss(train,'MasVnrType')
str_miss(train,'BsmtQual')
str_miss(train,'BsmtCond')
str_miss(train,'BsmtExposure')
str_miss(train,'BsmtFinType1')
str_miss(train,'BsmtFinType2')
str_miss(train,'Electrical')
str_miss(train,'FireplaceQu')
str_miss(train,'GarageType')
str_miss(train,'GarageFinish')
str_miss(train,'GarageQual')
str_miss(train,'GarageCond')
str_miss(train,'PoolQC')
str_miss(train,'Fence')
str_miss(train,'MiscFeature')

train.columns[train.isnull().any()].tolist()

**At this point all the missing values have been imputed, however outliers still remain which we can deal with by creating another function.**

In [ ]:
train.describe()

In [ ]:
#Function for Finding and replacing outliers with mean
def change_outlier(df_in, col_name):
    q1 = df_in[col_name].describe()['25%']
    q3 = df_in[col_name].describe()['75%']
    m = df_in[col_name].describe()['mean']
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-(1.5*iqr)
    fence_high = q3+(1.5*iqr)
    df_in.loc[(df_in[col_name] <= fence_low) | (df_in[col_name] >= fence_high),col_name] = m
    return df_in

the only columns that outliers need to be changed are as follows-
MasVnrArea, garageArea
*only these values included as garage area even for bigger houses is nearly the same however other areas are not outliers but represent larger houses and their corresponding sales values are large too*

In [ ]:
change_outlier(train,'GarageArea')
change_outlier(train,'MasVnrArea')
train.head()

**All the outliers and missing values have been taken care of and will now plot grpahs to find inferences but before that i'll transform the salesPrice with its log to make it more normal and reduce the skew**

In [ ]:
#log transform the target 
train["SalePriceLog"] = np.log1p(train["SalePrice"])

In [ ]:
print(train.iloc[:,-1].describe())
seaborn.distplot(train['SalePrice'])

In [ ]:
#qqplot
stats.probplot(train.iloc[:,-1],dist="norm",plot=pylab)
pylab.show

** seeing the qq plot we can clearly see that the SalesPriceLog is actually more normal than SalesPriceLog**

In [ ]:
#something else that i found online. It reports the skewness.
print("skewness of price: %f" % train['SalePrice'].skew())
print("skewness of log of price: %f" % train['SalePriceLog'].skew())

In [ ]:
#this gives a good idea on which numerical parameters does the salesprice depend on
train.corr().iloc[-1,:]

In [ ]:
numer = train._get_numeric_data()
numername = numer.columns.values.tolist()
for name in numername:
    plt.scatter(train.SalePrice,train[name])
    plt.legend()
    plt.show()

From these plots and correlation table one can conclude that the factors that affect Price most are OverallQual, GrLivArea, Basement area and 1st floor area. i.e the numerical quantities representing area and the overall quality

In [ ]:
allname = train.select_dtypes(include='object').columns.values.tolist()
for name in allname:
    train.boxplot(column = 'SalePrice', by = name)
    plt.show()

it again seems that condition parameters and central ac are more important than the others

**Starting with the model building now**

In [ ]:
targetLog = train.iloc[:,-1]
target = train.iloc[:,-2]
del train['SalePrice']
del train['SalePriceLog']
del train['Id']
train.shape
#now  train has all the predictors and target and targetLog have the fianl values

In [ ]:
#splitting train into train and test to see how well model works
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2)

#First Baseline Model with SalePrice as Predictor
predicted = y_train.mean()
size = y_test.size
sum =0;
for i in range(size):
    sum = sum + ((y_test.iloc[i] - predicted)*(y_test.iloc[i] - predicted))
mse = sum/size
rmse = mse**0.5
rmse

In [ ]:
#splitting train into train and test to see how well model works
X_train, X_test, y_train, y_test = train_test_split(train, targetLog, test_size=0.2)

#Second Baseline Model with SalePriceLog as Predictor
mean = y_train.mean()
predicted = np.expm1(mean)
size = y_test.size
sum =0;
for i in range(size):
    sum = sum + ((y_test.iloc[i] - predicted)*(y_test.iloc[i] - predicted))
mse = sum/size
rmse = mse**0.5
rmse

Two baseline models were created using SalePrice and SalePriceLog and it seems that SalePrice is doing better with lower root mean squared error!! I personally thought log would do better but in this case it didnt!!

**Moving to Linear Regression**

In [ ]:
#Dummy coding for categorical Variables
train = pd.get_dummies(train)
print(train.shape)

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

In [ ]:
#Making the first model
from sklearn.linear_model import LinearRegression
model1=LinearRegression()
model1.fit(X_train,y_train)
print("Mean squared error in Test:",np.mean((model1.predict(X_test) - y_test) ** 2))
print('R² of Test:',model1.score(X_test, y_test))
print("Mean squared error in train:",np.mean((model1.predict(X_train) - y_train) ** 2))
print('R² of train:',model1.score(X_train, y_train))

This seems a problem of overfitting as test model has 94% R2 but the test counterpart has has very low R2. Anyways, let us make the secong model and then we will look at the assumptions followed by better models

In [ ]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(train, targetLog, test_size=0.2)
model2=LinearRegression()
model2.fit(X_train_log,y_train_log)
print("Mean squared error in Test:",np.mean((model2.predict(X_test_log) - y_test_log) ** 2))
print('R² of Test:',model2.score(X_test_log, y_test_log))
print("Mean squared error in train:",np.mean((model2.predict(X_train_log) - y_train_log) ** 2))
print('R² of train:',model2.score(X_train_log, y_train_log))

This seems a much better model as the R2 on test is better and i feel it is because the SalePriceLog columnn is much more normal as seen from the qq plots above. This is a clear comparison that normal coulmns result in much better results!! :) (Ranges are there as it is selection biased and for every run there is a different R2)

**Time to check if assumptions were satisfied**.

In [ ]:
#Residual vs fitted plot for training set for second model
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = seaborn.residplot(model2.predict(X_train_log), y_train_log, lowess=True,scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted - Train Model 2')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')
plt.show()

#Residual vs fitted plot for test set for second model
plot_lm_2 = plt.figure(1)
plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0] = seaborn.residplot(model2.predict(X_test_log), y_test_log,lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_2.axes[0].set_title('Residuals vs Fitted - Test Model 2')
plot_lm_2.axes[0].set_xlabel('Fitted values')
plot_lm_2.axes[0].set_ylabel('Residuals')
plt.show()

In [ ]:
#Residual vs fitted plot for training set for first model
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = seaborn.residplot(model1.predict(X_train), y_train, lowess=True,scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted - Train Model 1')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')
plt.show()

#Residual vs fitted plot for test set for first model
plot_lm_2 = plt.figure(1)
plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0] = seaborn.residplot(model1.predict(X_test), y_test,lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_2.axes[0].set_title('Residuals vs Fitted - Test Model 1')
plot_lm_2.axes[0].set_xlabel('Fitted values')
plot_lm_2.axes[0].set_ylabel('Residuals')
plt.show()

Time for my analysis.
The first Model, made with SalePrice did not have a very normal qq plot and its residual vs fitted plot also has a parabolic structrure which shows non-linearity. All of this contributes to a not very good model.
However, the second model with SalePriceLog had a nearly normal qq plot and its residual vs fitted plot also has no pattern which means that all the assumptions of linear regression are satisfied. 

So uptill this point our best Model is a MultipleLinearRegression plot using SalePriceLog as the targeted value.


In both the models, There is a problem of overfitting as R2 for train is much higher than that for test.


**I will now be using Cross Vaidation to reduce overfitting as i feel that the Model fits the train model too well, i.e. i think there is a problem of overfitting.**

**CROSS VALIDATION FOR FEATURE SELECTION AND REDUCE OVERFITTING**

All right, i will quickly explain what i have done. I initially took a model that has all the columns as predictors and found its rmse. Now i remove 1 column at a time and again find rmse score using K- fold cross validation(here k is 10). If the score has increased then we were better off having that column, so i append it back into the included_cols list. After the code has run, i will have a list of all the predictors that contribute the best to Predicting correctly.

And the icing over the cake, it is not selection biased.

In [ ]:
from sklearn.model_selection import cross_val_score
lm  = LinearRegression()
score = (np.sqrt(cross_val_score(lm, train, targetLog, cv=10, scoring='neg_mean_squared_error') * -1)).mean()
print (score)

In [ ]:
all_cols = train.columns.values.tolist()
included_cols = train.columns.values.tolist()
for i in all_cols:
    prevscore = score
    included_cols.remove(i)
    score = (np.sqrt(cross_val_score(lm, train[included_cols], targetLog, cv=10, scoring='neg_mean_squared_error') * -1)).mean()
    if (score>prevscore):
        included_cols.append(i)
        print('reverted')

In [ ]:
final_score = (np.sqrt(cross_val_score(lm, train[included_cols], targetLog, cv=10, scoring='neg_mean_squared_error') * -1)).mean()
final_score

**The final score has reduced considerably, rather exponentially ;);). The score is much better than having all the predictors, therefore we can conclude that this model is better than the previous regression models and is also free from overfit!!**

In [ ]:
#List of all the columns to be used to train for LinearRegression
print (len(included_cols))
included_cols

In [ ]:
#Building the regression Model using the list of columns found using cross validation.
to_use = train[included_cols]
model3 = LinearRegression()
model3.fit(to_use,targetLog)
print("Root Mean squared error:",np.sqrt(np.mean((model3.predict(to_use) - targetLog) ** 2)))
print('R²:',model3.score(to_use, targetLog))

The R2 for this model is **89.35%** which is pretty good and its rmse is **0.1302955**. We will now try feature Selection techniques and build a model using them too. After thet we will compare and accept the best model

**RIDGE REGRESSION**

In [ ]:
from sklearn.linear_model import Ridge
## training the model
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2,0.5,0.05, 1, 5, 10, 20]
for i in alpha_ridge:
    ridgeReg = Ridge(alpha=i, normalize=True)
    ridgeReg.fit(train,targetLog)
    pred = ridgeReg.predict(train)
    print("Root Mean squared error for ",i," is:",np.sqrt(np.mean((pred - targetLog) ** 2)))

Minimum rmse is for alpha = 0.001.  Let us select that Ridge Regression Model.

In [ ]:
ridgeReg = Ridge(alpha=0.001, normalize=True)
ridgeReg.fit(train,targetLog)
pred = ridgeReg.predict(train)
print("Root Mean squared error is:",np.sqrt(np.mean((pred - targetLog) ** 2)))

**LASSO REGRESSION**

In [ ]:
from sklearn.linear_model import Lasso
lasso_ridge = [0.0005,1e-3,1e-2,0.5,0.05, 1, 5, 10, 20]
for i in lasso_ridge:
    lassoReg = Lasso(alpha=i, normalize=True)
    lassoReg.fit(train,targetLog)
    pred = lassoReg.predict(train)
    print("Root Mean squared error for ",i," is:",np.sqrt(np.mean((pred - targetLog) ** 2)))

Minimum rmse is for alpha = 0.0005.  Let us select that Lasso Regression Model.

In [ ]:
lassoReg = Lasso(alpha=0.0005, normalize=True)
lassoReg.fit(train,targetLog)
pred = lassoReg.predict(train)
print("Root Mean squared error is:",np.sqrt(np.mean((pred - targetLog) ** 2)))

**FINAL MODEL SELECTION**

The rmse of the LinearRegressionModel with Cross Validation was higher than the other two

Between Lasso and Ridge Regression, they have similar rmse, so i will choose Lasso Regression because when there are a large number f parameters, it is better to take Lasso as it cn actually make parameters = 0

In [ ]:
#inputting the files again and concatenating
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
del train['SalePrice']
print(train.shape)
print(test.shape)

frames = [test,train]
conc  = pd.concat(frames)

In [ ]:
test.head()

conc has all the rows of test and train combined, now i will remove all the missing values.

In [ ]:
#Finding columns with missing values and replacing using functions defined above
missing = conc.columns[conc.isnull().any()].tolist()
for i in missing:
    types = conc[i].dtype
    if (types == object):
        str_miss(conc,i)
    else:
        num_miss(conc, i)

In [ ]:
#Creating dummy variables
conc = pd.get_dummies(conc)
print(conc.shape)

In [ ]:
#Getting the test set back
test = conc.iloc[0:1459,:]
test.head()

In [ ]:
#Dropping the id column in test
del test['Id']
test.shape

In [ ]:
LogPredicted = lassoReg.predict(test)
Predicted = np.expm1(LogPredicted)
Predicted

***Predicted is the final answer that contains all the predicted values*** 

I am going to concatenate it to the test set.

In [ ]:
test = pd.read_csv("../input/test.csv")
test['PredictedSalePrice'] = list(Predicted)

In [ ]:
test.head()